In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

RAW_PATH = Path("../data/raw/ESTUDIO DE CIRUGIA CONSERVADORA.ods")
OUT_DIR = Path("../data/processed")
OUT_DIR.mkdir(parents=True, exist_ok=True)

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 120)


In [3]:
# Listar hojas disponibles
xls = pd.ExcelFile(RAW_PATH, engine="odf")
xls.sheet_names

['Hoja1']

In [ ]:
sheet = xls.sheet_names[0]  
df_raw = pd.read_excel(RAW_PATH, engine="odf", sheet_name=sheet)

df_raw.shape, df_raw.head()

((61, 26),
    EXPEDIENTE  EDAD LADO     ETAPA HISTOLOGIA     SUBTIPO  TAMAÑO T  GANGLIOS RESECADOS  GANGLIOS (+) QT NEO  \
 0    197025.0  58.0   1ZQ        I     DUCTAL  TRIPLE NEG       1.6                 0.0           0.0     NO   
 1    272467.0  47.0   DER  IN SITU     DUCTAL   LUMINAL A       0.6                 0.0           0.0     NO   
 2    279194.0  60.0   DER      IIB     DUCTAL   LUMINAL B       4.5                 NaN           NaN     SI   
 3    309706.0  56.0   DER      IIB     DUCTAL       BASAL       1.5                 6.0           1.0     SI   
 4    313432.0  51.0   IZQ     IIIC     DUCTAL       HER2+       3.0                17.0           4.0     SI   
 
    DOSIS DE RT FRACCIONAMIENTO BOOST ILV (+)   FECHA DX FECHA ULTIMA METS   SITIO  MESES METS  MESES VIGILANCIA  \
 0         50.0        ESTANDAR    NO      NO 2015-02-13   2025-05-27   NO       0         0.0             123.0   
 1         50.0        ESTANDAR    SI      NO 2017-08-01   2025-07-17   NO   

In [5]:
df_raw.info()
df_raw.isna().mean().sort_values(ascending=False).head(20)
df_raw.columns

<class 'pandas.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   EXPEDIENTE            60 non-null     float64       
 1   EDAD                  60 non-null     float64       
 2   LADO                  60 non-null     str           
 3   ETAPA                 60 non-null     str           
 4   HISTOLOGIA            60 non-null     str           
 5   SUBTIPO               60 non-null     str           
 6   TAMAÑO T              60 non-null     float64       
 7   GANGLIOS RESECADOS    59 non-null     float64       
 8   GANGLIOS (+)          59 non-null     float64       
 9   QT NEO                59 non-null     str           
 10  DOSIS DE RT           60 non-null     float64       
 11  FRACCIONAMIENTO       60 non-null     str           
 12  BOOST                 59 non-null     str           
 13  ILV (+)               59 non-null

Index(['EXPEDIENTE', 'EDAD', 'LADO ', 'ETAPA', 'HISTOLOGIA', 'SUBTIPO', 'TAMAÑO T', 'GANGLIOS RESECADOS',
       'GANGLIOS (+)', 'QT NEO', 'DOSIS DE RT', 'FRACCIONAMIENTO', 'BOOST', 'ILV (+)', 'FECHA DX', 'FECHA ULTIMA',
       'METS', 'SITIO', 'MESES METS', 'MESES VIGILANCIA', 'ESTADO ACTUAL', 'HORMONOTERAPIA', 'HORMONOTERAPIA CUAL',
       'QT ADYUVANTE', 'QT. TIPO', 'GANGLIOS RESECADOS.1'],
      dtype='str')

In [6]:
import re
import unicodedata

def normalize_colname(s: str) -> str:
    s = str(s).strip().lower()
    # quitar acentos y caracteres especiales (ñ -> n, á -> a, etc.)
    s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("ascii")
    # reemplazos comunes
    s = s.replace("(+)", "pos").replace("+", "pos")
    # snake_case
    s = re.sub(r"\s+", "_", s)
    s = re.sub(r"[^\w_]", "", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s

df = df_raw.copy()

# 1) normalizar columnas
df.columns = [normalize_colname(c) for c in df.columns]

# 2) renombre explícito (para que quede claro y consistente)
rename_map = {
    "expediente": "patient_id",
    "edad": "age",
    "lado": "side",
    "etapa": "stage",
    "histologia": "histology",
    "subtipo": "subtype",
    "tamano_t": "tumor_size_t",
    "ganglios_resecados": "nodes_removed",
    "ganglios_pos": "nodes_positive",
    "qt_neo": "neoadjuvant_chemo",
    "dosis_de_rt": "rt_dose",
    "fraccionamiento": "rt_fractionation",
    "boost": "rt_boost",
    "ilv_pos": "lvi_positive",                 # ILV(+): invasión linfovascular
    "fecha_dx": "dx_date",
    "fecha_ultima": "last_followup_date",
    "mets": "mets",
    "sitio": "mets_site",
    "meses_mets": "months_to_mets",
    "meses_vigilancia": "followup_months",
    "estado_actual": "current_status",
    "hormonoterapia": "hormone_therapy",
    "hormonoterapia_cual": "hormone_therapy_type",
    "qt_adyuvante": "adjuvant_chemo",
    "qt_tipo": "chemo_type",
}

df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})

# 3) quitar filas/columnas totalmente vacías
df = df.dropna(axis=0, how="all").dropna(axis=1, how="all")

df.columns


Index(['patient_id', 'age', 'side', 'stage', 'histology', 'subtype', 'tumor_size_t', 'nodes_removed', 'nodes_positive',
       'neoadjuvant_chemo', 'rt_dose', 'rt_fractionation', 'rt_boost', 'lvi_positive', 'dx_date', 'last_followup_date',
       'mets', 'mets_site', 'months_to_mets', 'followup_months', 'current_status', 'hormone_therapy',
       'hormone_therapy_type', 'adjuvant_chemo', 'chemo_type', 'ganglios_resecados1'],
      dtype='str')

In [7]:
[c for c in df.columns if "nodes_removed" in c or "ganglios_resecados" in c]


['nodes_removed', 'ganglios_resecados1']

In [8]:
# fusionar columnas duplicadas de ganglios resecados
df["nodes_removed"] = df["nodes_removed"].fillna(df["ganglios_resecados1"])

# eliminar la columna duplicada
df = df.drop(columns=["ganglios_resecados1"])

# verificación
df["nodes_removed"].isna().sum(), df.columns


(np.int64(1),
 Index(['patient_id', 'age', 'side', 'stage', 'histology', 'subtype', 'tumor_size_t', 'nodes_removed', 'nodes_positive',
        'neoadjuvant_chemo', 'rt_dose', 'rt_fractionation', 'rt_boost', 'lvi_positive', 'dx_date', 'last_followup_date',
        'mets', 'mets_site', 'months_to_mets', 'followup_months', 'current_status', 'hormone_therapy',
        'hormone_therapy_type', 'adjuvant_chemo', 'chemo_type'],
       dtype='str'))

In [9]:
# tipos numéricos
numeric_cols = [
    "age",
    "tumor_size_t",
    "nodes_removed",
    "nodes_positive",
    "months_to_mets",
    "followup_months",
]

for c in numeric_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# fechas (por seguridad)
date_cols = ["dx_date", "last_followup_date"]
for c in date_cols:
    if c in df.columns:
        df[c] = pd.to_datetime(df[c], errors="coerce")

df.info()


<class 'pandas.DataFrame'>
Index: 60 entries, 0 to 60
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   patient_id            60 non-null     float64       
 1   age                   60 non-null     float64       
 2   side                  60 non-null     str           
 3   stage                 60 non-null     str           
 4   histology             60 non-null     str           
 5   subtype               60 non-null     str           
 6   tumor_size_t          60 non-null     float64       
 7   nodes_removed         59 non-null     float64       
 8   nodes_positive        59 non-null     float64       
 9   neoadjuvant_chemo     59 non-null     str           
 10  rt_dose               60 non-null     float64       
 11  rt_fractionation      60 non-null     str           
 12  rt_boost              59 non-null     str           
 13  lvi_positive          59 non-null     

In [11]:
from pathlib import Path

OUT_DIR = Path("../data/processed")
OUT_DIR.mkdir(parents=True, exist_ok=True)

df.to_csv(
    OUT_DIR / "breast_cancer_clean_v0.csv",
    index=False,
    encoding="utf-8"
)
